In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

## 단어 임베딩 사용하기
* 단어와 벡터를 연관짓는 강력하고 인기 있는 또 다른 방법은 단어 임베딩이라는 밀집 단어 벡터를 사용하는 것임
* 원-핫 인코딩으로 만든 벡터는 희소하고(대부분 0으로 채워집니다) 고차원임(어휘 사전에 있는 단어의 수와 차원이 같음
* 반면 단어 임베딩은 저차원의 실수형 벡터임(희소 벡터의 반대인 밀집 벡터임)
* 원-핫 인코딩으로 얻은 단어 벡터와 달리 단어 임베딩은 데이터로부터 학습됨
* 보통 256차원, 512차원 또는 큰 어휘 사전을 다룰 때는 1,024차원의 단어 임베딩을 사용함
* 반면 원-핫 인코딩은 (20,000개의 토큰으로 이루어진 어휘 사전을 만들려면) 20,000차원 또는 그 이상의 벡터일 경우가 많음
* 따라서 단어 임베딩이 더 많은 정보를 적은 차원에 저장함

![word embeddings vs. one hot encoding](https://s3.amazonaws.com/book.keras.io/img/ch6/word_embeddings.png)

## 단어 임베딩을 만드는 방법 두 가지
* (문서 분류나 감성 예측과 같은) 관심 대상인 문제와 함께 단어 임베딩을 학습합니다. 이런 경우에는 랜덤한 단어 벡터로 시작해서 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습합니다.
* 풀려는 문제가 아니고 다른 머신 러닝 작업에서 미리 계산된 단어 임베딩을 로드합니다. 이를 사전 훈련된 단어 임베딩이라고 합니다.

### `Embedding` 층을 사용해 단어 임베딩 학습하기
* `Embedding` 층을 (특정 단어를 나타내는) 정수 인덱스를 밀집 벡터로 매핑하는 딕셔너리로 이해하면됨
* `Embedding` 층은 크기가 `(samples, sequence_length)`인 2D 정수 텐서를 입력으로 받음
* 배치에 있는 모든 시퀀스는 길이가 같아야 하므로(하나의 텐서에 담아야 하기 때문에) 작은 길이의 시퀀스는 0으로 패딩되고 길이가 더 긴 시퀀스는 잘림
* `Embedding` 층은 크기가 `(samples, sequence_length, embedding_dimensionality)`인 3D 실수형 텐서를 반환함
* `Embedding` 층의 객체를 생성할 때 가중치(토큰 벡터를 위한 내부 딕셔너리)는 다른 층과 마찬가지로 랜덤하게 초기화되며, 훈련하면서 이 단어 벡터는 역전파를 통해 점차 조정되어 이어지는 모델이 사용할 수 있도록 임베팅 공간을 구성하고, 훈련이 끝나면 임베딩 공간은 특정 문제에 특화된 구조를 가지게 됨

In [2]:
from keras.layers import Embedding

# Embedding 층은 적어도 두 개의 매개변수를 받습니다.
# 가능한 토큰의 개수(여기서는 1,000으로 단어 인덱스 최댓값 + 1입니다)와 임베딩 차원(여기서는 64)입니다
embedding_layer = Embedding(1000, 64)  # input_dim, output_dim

### 익숙한 IMDB 영화 리뷰 감성 예측 문제에 적용해 보자
* 영화 리뷰에서 가장 빈도가 높은 10,000개의 단어를 추출하고 리뷰에서 20개 단어 이후는 버림
* 이 네트워크는 10,000개의 단어에 대해 8 차원의 임베딩을 학습하여 정수 시퀀스 입력(2D 정수 텐서)를 임베딩 시퀀스(3D 실수형 텐서)로 바꿀 것임
* 그 다음 이 텐서를 2D로 펼쳐서 분류를 위한 `Dense` 층을 훈련할 것임

In [3]:
from keras.datasets import imdb
from keras import preprocessing

In [4]:
# 특성으로 사용할 단어의 수
max_features = 10000
# 사용할 텍스트의 길이(가장 빈번한 max_features 개의 단어만 사용합니다)
maxlen = 20

# 정수 리스트로 데이터를 로드합니다.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(x_train[:1])

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])]


In [5]:
# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환합니다.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)  # reverse 추출
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print(x_train[:1])  

[[  65   16   38 1334   88   12   16  283    5   16 4472  113  103   32
    15   16 5345   19  178   32]]


In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding 층에 input_length를 지정합니다.
model.add(Embedding(10000, 8, input_length=maxlen))  # 10000, 8, 20
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됩니다.

# 3D 임베딩 텐서를 (samples, maxlen * 8) 크기의 2D 텐서로 펼칩니다.
model.add(Flatten())

# 분류기를 추가합니다.
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                5152      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 85,185
Trainable params: 85,185
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)




Train on 20000 samples, validate on 5000 samples
Epoch 1/10





20000/20000 [==============================] - 1s 74us/step - loss: 0.6149 - acc: 0.6526 - val_loss: 0.5227 - val_acc: 0.7352
Epoch 2/10
20000/20000 [==============================] - 2s 76us/step - loss: 0.4494 - acc: 0.7870 - val_loss: 0.4991 - val_acc: 0.7538
Epoch 3/10
20000/20000 [==============================] - 1s 72us/step - loss: 0.3904 - acc: 0.8253 - val_loss: 0.5031 - val_acc: 0.7538
Epoch 4/10
20000/20000 [==============================] - 2s 75us/step - loss: 0.3451 - acc: 0.8509 - val_loss: 0.5234 - val_acc: 0.7456
Epoch 5/10
20000/20000 [==============================] - 1s 62us/step - loss: 0.3034 - acc: 0.8735 - val_loss: 0.5524 - val_acc: 0.7446
Epoch 6/10
20000/20000 [==============================] - 1s 67us/step - loss: 0.2638 - acc: 0.8948 - val_loss: 0.5906 - val_acc: 0.7370
Epoch 7/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.2270 - acc: 0.9127 - val_loss: 0.6420 - 

* 약 70% 초반 정도의 검증 정확도가 나옴
* 리뷰에서 20개의 단어만 사용한 것치고 꽤 좋은 결과이지만, 임베딩 시퀀스를 펼치고 하나의 Dense 층을 훈련했으므로 입력 시퀀스에 있는 각 단어를 독립적으로 다루었음
* 단어 사이의 관계나 문장의 구조를 고려하지 않았음(예를 들어 이 모델은 “this movie is a bomb”와 “this movie is the bomb”를 부정적인 리뷰로 동일하게 다룰 것임)
* 각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환 층이나 1D 합성곱 층을 추가하는 것이 좋을 것임

In [28]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 0s 15us/step


[0.8328652380943299, 0.71188]

In [30]:
print(history.epoch)
print(history.params)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
{'batch_size': 32, 'epochs': 10, 'steps': None, 'samples': 20000, 'verbose': 1, 'do_validation': True, 'metrics': ['loss', 'acc', 'val_loss', 'val_acc']}
